In [1]:
import os
os.chdir("../")
%pwd

'e:\\DataScienceProjects\\car-price-prediction'

In [2]:
from dataclasses import dataclass
from pathlib import Path
from typing import List

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_file_path: Path
    model_state_file_path: Path
    embed_dim_file_path: Path
    num_dim_file_path: Path
    data_file_path: Path
    evaluation_results: Path
    batch_size: Path
    layers: List
    dropout: float

In [3]:

from carPricePrediction.constants import *
from carPricePrediction.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.EvaluationArguments

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_file_path=config.model_file_path,
            model_state_file_path = config.model_state_file_path,
            embed_dim_file_path = config.embed_dim_file_path,
            num_dim_file_path = config.num_dim_file_path,
            data_file_path = config.data_file_path,
            evaluation_results = config.evaluation_results,
            batch_size = params.batch_size,
            layers = params.layers,
            dropout =  params.layers
        )

        return model_evaluation_config

In [4]:
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from carPricePrediction.components.feed_forward_nn import FeedForwardNN

class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def evaluate_model(self):
        # Set device
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        
        # Load test data
        test_data = torch.load(self.config.data_file_path)
        # Create data loader
        test_loader = DataLoader(test_data, batch_size=self.config.batch_size, shuffle=True)
        
        # Load dimensions
        with open(self.config.num_dim_file_path, 'rb') as f:
            num_dim = pickle.load(f)
        with open(self.config.embed_dim_file_path, 'rb') as f:
            embedding_dim = pickle.load(f)

        # Load trained model
        if torch.cuda.is_available():
            # Load the entire model
            model = torch.load(self.config.model_file_path)
        else:
            # Recreate the model architecture
            model = FeedForwardNN(
                embedding_dim=embedding_dim, 
                n_cont=num_dim, 
                out_dim=1, 
                layers=self.config.layers, 
                dout=self.config.dropout
            )
            
            # Load the state dictionary into the model
            model.load_state_dict(torch.load(self.config.model_state_file_path))
        
            # Set it to current device (not needed)
            model.to(device)

        # Set loss function
        loss_function = nn.MSELoss()

        model.eval()  # Set the model to evaluation mode
        test_loss = 0

        # Model Evaluation
        with torch.no_grad():
            for cat_data, num_data, target in test_loader:
                cat_data, num_data, target = cat_data.to(device), num_data.to(device), target.to(device)
                prediction = model(cat_data, num_data)
                loss = torch.sqrt(loss_function(prediction, target))
                test_loss += loss
            test_loss /= len(test_loader)

        # Save test loss
        results_path = os.path.join(self.config.evaluation_results, 'evaluation_results.txt')
        with open(results_path, 'w') as file:
            file.write(f'Test Loss (RMSE): {test_loss:.4f}')

In [5]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.evaluate_model()
except Exception as e:
    raise e

[2024-03-30 21:52:02,772: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2024-03-30 21:52:02,775: INFO: common: yaml file: params.yaml loaded successfully.]
[2024-03-30 21:52:02,777: INFO: common: Created directory at: artifacts]
[2024-03-30 21:52:02,778: INFO: common: Created directory at: artifacts/model]
Test Loss: 7755.7007
